In [1]:
#Import Libraries

In [2]:
!pip install shap

In [3]:
import tensorflow as tf
import statsmodels as sm
import pandas as pd
import numpy as np
import sklearn as skl
import numpy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization

from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score as R2
from sklearn.metrics import mean_absolute_percentage_error as MAPE

import shap


In [4]:
def RMSE(y_true, y_pred):
    return np.sqrt(MSE(y_true, y_pred))

### Preparing data

In [5]:
path_ds1 = "Krafthack2022/input_dataset-1.csv"
path_ds2 = "Krafthack2022/input_dataset-2.csv"
path_pred = "Krafthack2022/prediction.csv"

In [6]:
ds2 = pd.read_csv(path_ds2)

In [7]:
ds1 = pd.read_csv(path_ds1)

In [123]:
X_new = pd.read_csv(path_pred)

In [82]:
frames = [ds1, ds2]
# full_df = pd.concat(frames).sample(100000)
full_df = pd.concat(frames)

In [83]:
full_df = full_df.drop(['lower_bearing_vib_vrt', 'turbine_bearing_vib_vrt'], axis=1)

In [84]:
full_df["timepoints"] = pd.to_datetime(full_df["timepoints"], format='%Y-%m-%d %H:%M:%S') 

In [85]:
copy_df = full_df.copy()

In [86]:
copy_df.index = copy_df.timepoints
copy_df = copy_df.drop("timepoints", axis=1)

In [87]:
copy_df = copy_df.sort_index()

copy_df['seconds'] = copy_df.index - copy_df.index[0]

copy_df['seconds'] = copy_df['seconds'].dt.total_seconds()


#### Dealing with NaN values
In order to keep it simple we simply discarded rows of data with NaN values.

An alternative approach would be to use for example K-nearest neighbors, with the sklearn
KNNImputer, in this case you would ideally make a train/validation set for this specific purpose in order to choose n_neighbors. In order to avoid data leakage imputing should only be done on a training set,
so it shouldn't solve NaN problems in the hold-out set.

In [88]:
copy_df = copy_df.dropna()

# imputer = KNNImputer(n_neighbors=2)
# imputer.fit_transform(X)


In [89]:
x_cols = list(X_new.drop("timepoints", axis=1).columns)
x_cols.append("seconds")

In [90]:
#Y columns
y_cols = [c for c in copy_df.columns if "Tensile" in c]
Y_df = copy_df[y_cols]

#X columns
X_df = copy_df[x_cols]

In [91]:
#Cleaning up dataset

In [92]:
# plt.figure(figsize=(10,7))
# figure=sns.heatmap(X_df.corr(), annot=True)

##### Removed feature
We remove the following feature as it is 99% linearly correlated with another feature, so unless there is some very hidden non-linear correlation between the two, this should be fine.
Specifically Turbine_Guide Vane Opening is highly correlated with Unit_4_Power.

In [93]:
#Dropping column: Unit_4_Power
X_df = X_df.drop("Turbine_Guide Vane Opening",axis=1)

##### Adding different time intervals

In order to gauge whether there is some repeated (seasonal) patterns (e.g every monday the turbines are started) we add various time-period columns.

In [95]:
X_df["day"] = X_df.index.day
X_df["weekday"] = X_df.index.weekday
X_df["month"] = X_df.index.month
X_df["hour"] = X_df.index.hour

X_df_ = X_df.copy()

In [96]:
X_df = pd.get_dummies(X_df_, columns=["mode"], drop_first=True)


#### Splitting data and scaling the data
We split the data in train, validation and test.
We use the validation set for early stopping and hyperparameter tuning.
The train and validation are shuffled. However, we keep the test set non-shuffled, not because the temporal order matter (this is not a time-series model), but because we want nice plots with no time-gaps (in a production scenario we wouldn't do it like this).

We scale the data to the interval [0, 1]

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X_df, Y_df, test_size=0.2, shuffle=False)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1, shuffle=True) # 0.25 x 0.8 = 0.2

cols_scale = [c for c in X_train.columns if c != 'mode_start']
scaler = MinMaxScaler()
scaler.fit(X_train.loc[:, cols_scale])


X_train.loc[:, cols_scale] = scaler.transform(X_train.loc[:, cols_scale])
X_test.loc[:, cols_scale] = scaler.transform(X_test.loc[:, cols_scale])
X_val.loc[:, cols_scale] = scaler.transform(X_val.loc[:, cols_scale])


['Unit_4_Power', 'Unit_4_Reactive Power', 'Turbine_Pressure Drafttube', 'Turbine_Pressure Spiral Casing', 'Turbine_Rotational Speed', 'seconds', 'day', 'weekday', 'month', 'hour']



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [98]:
# sample_size = 10**5
# X_train = X_train.sample(sample_size)
# y_train = y_train.loc[X_train.index, :]

# X_val = X_train.sample(sample_size)
# y_val = y_train.loc[X_train.index, :]

In [99]:
class MCDropout(Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [100]:
class NN_Model():
    def __init__(self, X_train, y_train, X_val, y_val, X_test, y_test):
        self.X_train, self.y_train = X_train, y_train
        self.X_val, self.y_val = X_val, y_val
        self.X_test, self.y_test = X_test, y_test
        
        self.model = None
        self.pred_model = None
        
        self.features_num = self.X_train.shape[1]
        self.targets_num = self.y_train.shape[1]
        
    def make_nn_model(self, layers=None,
                      activation='relu', drop_rate=0.2, drop_rate_inference=False):
        
        if not drop_rate_inference:
            self.layers, self.activation = layers, activation
            dropout = Dropout
        else:
            layers, activation = self.layers, self.activation
            #Using dropout during prediction for uncertainty
            dropout = MCDropout
            
        inputs = Input(shape=(self.features_num,))
        for i, lay in enumerate(layers):
            if i == 0:
                x = Dense(lay, activation='relu')(inputs)
#                 x = BatchNormalization()(x)
#                 x = dropout(drop_rate)(x)
            else:
                x = Dense(lay, activation='relu')(x)
#                 x = BatchNormalization()(x)
#                 x = dropout(drop_rate)(x)
        outputs = Dense(self.targets_num, activation='linear')(x)
        model = tf.keras.Model(inputs=inputs, outputs=outputs, name="Evens_lille_venn")
        if not drop_rate_inference:
#             model.summary()
            self.model = model
        else:
            return model
        
    def for_tune(self, layers, activation, drop_rate, lr, epochs=4000, patience=10, batch_size=32, verbose=0):
        tf.keras.backend.clear_session()
        self.make_nn_model(layers, activation, drop_rate)
        self.fitting(lr=lr, batch_size=batch_size, epochs=epochs, patience=patience, verbose=verbose)
        
        y_pred = self.model.predict(X_val)
        
        return y_pred
        
    
    def fitting(self, epochs=1000, patience=100, batch_size=32, lr=0.00001, verbose=1, save_path=None):
        if self.model is not None:
            self.epochs, self.patience, self.batch_size, self.lr = epochs, patience, batch_size, lr
            es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)
            optimizer = tf.keras.optimizers.Adam(lr=lr)
            self.model.compile(optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
            # Prepare the training dataset
            train_dataset = tf.data.Dataset.from_tensor_slices((self.X_train, self.y_train))
            train_dataset = train_dataset.shuffle(buffer_size=4096).batch(batch_size)

            # Prepare the validation dataset
            val_dataset = tf.data.Dataset.from_tensor_slices((self.X_val, self.y_val))
            val_dataset = val_dataset.batch(batch_size)
            self.history = self.model.fit(train_dataset, validation_data=val_dataset, epochs=epochs, callbacks=[es_callback], verbose=verbose)
            if save_path is not None:
                self.model.save_weights(save_path)
            return self.model, self.history
        
    def fit_on_all(self, epochs=1000, patience=100, batch_size=32, lr=0.00001, verbose=1, save_path=None):
        if self.model is not None:
            self.epochs, self.patience, self.batch_size, self.lr = epochs, patience, batch_size, lr
            es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)
            optimizer = tf.keras.optimizers.Adam(lr=lr)
            self.model.compile(optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
            # Prepare the training dataset
            X_train = pd.concat([self.X_train, self.X_test])
            y_train = pd.concat([self.y_train, self.y_test])
            train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
            train_dataset = train_dataset.shuffle(buffer_size=4096).batch(batch_size)

            # Prepare the validation dataset
            val_dataset = tf.data.Dataset.from_tensor_slices((self.X_val, self.y_val))
            val_dataset = val_dataset.batch(batch_size)
            self.history = self.model.fit(train_dataset, validation_data=val_dataset, epochs=epochs, callbacks=[es_callback], verbose=verbose)
            if save_path is not None:
                self.model.save_weights(save_path)
            return self.model, self.history        
        
    def load_model(self, path):
        self.model.load_weights(path)
#         for i, weights in enumerate(weights_list[0:9]):
#             model.layers[i].set_weights(weights)
    
    def dropout_model(self, drop_rate_predict=0.2):
        self.model_dropout = self.make_nn_model(drop_rate = drop_rate_predict, drop_rate_inference=True)
        self.model_dropout.set_weights(self.model.get_weights())
        return self.model_dropout
        
    def loss_plot(self):
        hist = self.history.history
        fig, ax = plt.subplots(figsize=(10, 15))
        ax.plot(np.arange(len(hist['val_loss'])), hist['val_loss'], label='val_loss')
        ax.plot(np.arange(len(hist['val_loss'])), hist['loss'], label='loss')
        plt.legend()

In [101]:
def dropout_predict(model, X, y, credible_interval=0.95, iterations=100):
    if iterations < 2:
        iterations = 2
    predictions = np.zeros((y.shape[0], y.shape[1], iterations))
    for i in range(iterations):
        pred = model.predict(X)
        predictions[:, :, i] = pred
    y_pred = np.mean(predictions, axis=2)
    lower = np.quantile(predictions, 0.5-credible_interval/2, axis=2)
    upper = np.quantile(predictions, 0.5-credible_interval/2, axis=2)
    
    return y_pred, lower, upper

In [102]:

def random_grid_tuner(save_path='test'):
    layers_ls = [[32, 32], [32, 32, 32], [32, 32, 32, 32], [64, 64, 64, 64], [128, 128, 128, 128]]
    lr_ls = [0.001, 0.0005, 0.0001, 0.00001]
    drop_rate_ls = [0.1, 0.2, 0.3, 0.4]
    activation_ls = ['relu', 'swish']
    batch_size_ls = [4096]

    grid = {'layers':layers_ls, 'lr':lr_ls, 'drop_rate':drop_rate_ls, 'activation':activation_ls, 'batch_size': batch_size_ls}

    ix_ls = []

    mape_best = 100
    rounds = 5
    np.random.seed(43)
    for i in range(rounds):
        print("")
        print("Round: ", i)
        round = 0
        while True:
            ixs = []
            use_vals = {}
            for key, val in grid.items():
                ix = np.random.randint(0, high=len(val))
                ixs.append(ix)
                use_vals[key] = val[ix]
            again = False
            for ix_in in ix_ls:
                if ixs == ix_in:
                    again = True
            if not again:
                break
            else:
                round += 1
            if round == 3:
                break
        print("Using params: ", use_vals)
        y_val_pred = nn.for_tune(**use_vals, verbose=2)
        mape = MAPE(y_val, y_val_pred)
        r2 = R2(y_val, y_val_pred)
        print("MAPE: ", mape)
        print("R2: ", r2)
        if mape < mape_best:
            mape_best = mape
            best_model = nn.model
            best_params = use_vals
            print("")
            print("***New Best***")
            print("Best params: ", best_params)
            print("Best MAPE: ", mape_best)
            print("Best r2: ", r2)

    print("mape_best", mape_best)
    print("best_params", best_params)
    best_model.save_weights(save_path)
    return best_model, mape_best


In [103]:
nn = NN_Model(X_train, y_train, X_val, y_val, X_test, y_test)
nn.make_nn_model(layers=[20, 20, 20], drop_rate=0.05, activation='relu')
# nn.make_nn_model(layers=[20, 20, 20], drop_rate=0.3, activation='relu')

# best_model, mape_best = random_grid_tuner()


In [104]:
# nn.model.save_weights('temp_model')

In [105]:
# nn.load_model('temp_model_2')
nn.load_model('good_model')

# 
nn.fit_on_all(lr=0.00001, batch_size=128, epochs=50, patience=8)
nn.model.save_weights('final_model')
# nn.fitting(lr=0.00001, batch_size=32, epochs=200, patience=5, save_path='temp_model_3')

Epoch 1/50
64735/64735 [==============================] - 42s 648us/step - loss: 257.1287 - root_mean_squared_error: 16.0351 - val_loss: 2765.3699 - val_root_mean_squared_error: 52.5868
Epoch 2/50
64735/64735 [==============================] - 43s 664us/step - loss: 83.8858 - root_mean_squared_error: 9.1590 - val_loss: 2024.0580 - val_root_mean_squared_error: 44.9896
Epoch 3/50
64735/64735 [==============================] - 43s 671us/step - loss: 66.1663 - root_mean_squared_error: 8.1344 - val_loss: 1669.6399 - val_root_mean_squared_error: 40.8613
Epoch 4/50
64735/64735 [==============================] - 120s 2ms/step - loss: 54.7521 - root_mean_squared_error: 7.3994 - val_loss: 1668.0266 - val_root_mean_squared_error: 40.8415
Epoch 5/50
64735/64735 [==============================] - 54s 841us/step - loss: 51.4238 - root_mean_squared_error: 7.1709 - val_loss: 1718.3999 - val_root_mean_squared_error: 41.4536
Epoch 6/50
64735/64735 [==============================] - 43s 659us/step - loss

(<tensorflow.python.keras.engine.functional.Functional at 0x1d84e104dc0>,
 <tensorflow.python.keras.callbacks.History at 0x1d9187b52e0>)

In [108]:
# nn.model.save_weights('final_model')

In [110]:
nn.load_model('final_model')
nn.fitting(lr=0.00001, batch_size=512, epochs=50, patience=20, save_path='final_model_2')

Epoch 1/50
12138/12138 [==============================] - 42s 3ms/step - loss: 46.4968 - root_mean_squared_error: 6.8189 - val_loss: 7.1538 - val_root_mean_squared_error: 2.6746
Epoch 2/50
12138/12138 [==============================] - 41s 3ms/step - loss: 7.1248 - root_mean_squared_error: 2.6692 - val_loss: 7.0863 - val_root_mean_squared_error: 2.6620
Epoch 3/50
12138/12138 [==============================] - 42s 3ms/step - loss: 7.0748 - root_mean_squared_error: 2.6598 - val_loss: 7.0462 - val_root_mean_squared_error: 2.6545
Epoch 4/50
12138/12138 [==============================] - 38s 3ms/step - loss: 7.0440 - root_mean_squared_error: 2.6540 - val_loss: 7.0228 - val_root_mean_squared_error: 2.6501
Epoch 5/50
12138/12138 [==============================] - 42s 3ms/step - loss: 7.0252 - root_mean_squared_error: 2.6505 - val_loss: 7.0068 - val_root_mean_squared_error: 2.6470
Epoch 6/50
12138/12138 [==============================] - 41s 3ms/step - loss: 7.0129 - root_mean_squared_error: 2

12138/12138 [==============================] - 31s 3ms/step - loss: 6.7750 - root_mean_squared_error: 2.6029 - val_loss: 6.7623 - val_root_mean_squared_error: 2.6004
Epoch 48/50
12138/12138 [==============================] - 31s 3ms/step - loss: 6.7583 - root_mean_squared_error: 2.5997 - val_loss: 6.7450 - val_root_mean_squared_error: 2.5971
Epoch 49/50
12138/12138 [==============================] - 33s 3ms/step - loss: 6.7356 - root_mean_squared_error: 2.5953 - val_loss: 6.7262 - val_root_mean_squared_error: 2.5935
Epoch 50/50
12138/12138 [==============================] - 31s 3ms/step - loss: 6.7054 - root_mean_squared_error: 2.5895 - val_loss: 6.6838 - val_root_mean_squared_error: 2.5853


(<tensorflow.python.keras.engine.functional.Functional at 0x1d84e104dc0>,
 <tensorflow.python.keras.callbacks.History at 0x1d843479880>)

In [136]:
nn.load_model('final_model_2')

In [124]:
X_new = pd.get_dummies(X_new, columns=["mode"], drop_first=True)

In [125]:
X_new["timepoints"] = pd.to_datetime(X_new["timepoints"], format='%Y-%m-%d %H:%M:%S') 
X_new.index = X_new.timepoints
X_new = X_new.drop("timepoints", axis=1)

X_new = X_new.sort_index()

X_new['seconds'] = X_new.index - copy_df.index[0]

X_new['seconds'] = X_new['seconds'].dt.total_seconds()

X_new["day"] = X_new.index.day
X_new["weekday"] = X_new.index.weekday
X_new["month"] = X_new.index.month
X_new["hour"] = X_new.index.hour

cols_scale = [c for c in X_new.columns if c != 'mode_start']
scaler = MinMaxScaler()
scaler.fit(X_new.loc[:, cols_scale])


X_new.loc[:, cols_scale] = scaler.transform(X_new.loc[:, cols_scale])


In [130]:
X_new = X_new.drop('Turbine_Guide Vane Opening', axis=1)
X_new = X_new[X_test.columns]

In [140]:
y_pred_new = nn.model.predict(X_new)

In [142]:
y_pred_new_df = pd.DataFrame(y_pred, columns=y_test.columns)

In [144]:
y_pred_new_df.index = X_new.index

In [147]:
y_pred_new_df.to_csv('solution.csv')

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(15, 15))
axs = axs.flatten()

# y_test_sort = y_test.sort_index()
# X_test_sort = X_test.sort_index()

for i, ax in enumerate(axs):
    rmse = RMSE(y_test.iloc[:, i], y_pred[:, i])
    r2 = R2(y_test.iloc[:, i], y_pred[:, i])
    mape = MAPE(y_test.iloc[:, i], y_pred[:, i])
    ax.scatter(y_test.iloc[:, i], y_pred[:, i], label='r2: %s, mape %s' %(np.round(r2, 5), np.round(mape, 5)))
    ax.set_xlabel(r'y_test ($\mu m / m $)', fontsize=10)
    ax.set_ylabel(r'y_pred ($\mu m / m $)', fontsize=10)
    ax.set_title('Tensil %s' %(i))
    ax.legend(loc=2)

In [ ]:
# import numpy as np

# model, history = fitting(model, X_train, y_train, X_val, y_val, epochs=1000, patience=10, batch_size=32, lr=0.00001)

# hist = history.history
